In [20]:
import ccxt
import requests
from binance.client import Client
from binance.exceptions import BinanceAPIException
from datetime import datetime, timedelta
import time
import hashlib
import hmac
import json
import logging
from tqdm import tqdm

In [25]:
from main import *

In [26]:
def get_min_notional(symbol):
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    data = response.json()
    for s in data['symbols']:
        if s['symbol'] == symbol:
            for filter in s['filters']:
                if filter['filterType'] == 'MIN_NOTIONAL':
                    return float(filter['minNotional'])
    return None

In [35]:
symbol="XRPUSDT"

In [36]:
api_key, api_secret = get_keys("binance_keys.json")
client = Client(api_key, api_secret, testnet=True)

ticker = client.get_symbol_ticker(symbol=symbol)
current_price = float(ticker['price'])

In [37]:
current_price

0.4156

In [38]:
def calculate_and_place_order(symbol, old_delta):
    try:
        
        old_delta = float(old_delta)
        new_delta = float(information_for_options(client)[-2])
        contract_size = float(information_for_options(client)[-1])
        share_to_purchase = (new_delta - old_delta) * contract_size

        api_key, api_secret = get_keys("binance_keys.json")
        client = Client(api_key, api_secret, testnet=True)

        ticker = client.get_symbol_ticker(symbol=symbol)
        current_price = float(ticker['price'])
        logger.info(f"Current price of {symbol}: {current_price}")
        print(f"Current price of {symbol}: {current_price}")
        logger.info(f"Share to purchase: {share_to_purchase}")

        notional_value = current_price * share_to_purchase
        print(f"The purchase order in USD is {notional_value}")
        if notional_value >= 5.1:
            if share_to_purchase != 0:  # Only place an order if there's a change in delta
                min_qty, _ = get_lot_size(symbol)
                if abs(share_to_purchase) < min_qty:
                    logger.warning(f"Share to purchase too small: {share_to_purchase}")
                else:
                    side = "BUY" if new_delta > old_delta else "SELL"
                    print(f"Create {side} hedging order for: {symbol} for {round(abs(share_to_purchase), 6)}")
                    create_hedging_order(symbol, side, round(abs(share_to_purchase), 6))
            else:
                logger.info(f"{datetime.now()}: No change in delta. No order placed.")
                print("No Order placed no change in Delta")
        else: 
            logger.warning(f"Notional price: {notional_value} too low cant place order")
    except Exception as e:
        logger.error(f"Error calculating and placing order: {e}")
        raise Exception(f"An error occurred: {e}")

In [40]:
calculate_and_place_order(symbol,0)

NameError: name 'logger' is not defined

In [33]:
print(get_min_notional("XRPUSDT"))

None
